In [ ]:
import cantera as ct
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import matplotlib
import os
%matplotlib inline
from scipy.ndimage import gaussian_filter

In [ ]:
def add_lines_to_contourf(contour_obj):
    for col in  contour_obj.collections:
        col.set_linewidth(1.5)
        col.set_edgecolor(col.get_facecolor())

In [ ]:
def get_reaction_branching(solution, initial_reactants):
    """given a solution object at a specified temperature
    and pressure, finds the reaction rates of reactions
    corresponding to the initial_reactants, in both forward
    and reverse directions.
    
    returns a pandas.Series with the reaction index as its
    index and the reaction rate constant as its product.
    
    All reactants must be specified for a match to be found."""
    reactions = solution.reactions()
    reaction_rates = {}
    for index in range(len(reactions)):
        rxn = reactions[index]
        reactants = set(rxn.reactants.keys())
        products = set(rxn.products.keys())
        if reactants == initial_reactants:
            #print(rxn)
            reaction_rates[index] = solution.forward_rate_constants[index]
        elif products == initial_reactants:
            #print(rxn)
            reaction_rates[index] = solution.reverse_rate_constants[index]
        else:
            continue
    if len(reaction_rates) == 0:
        raise Exception('No reaction branching found for {0}'.format(initial_reactants))
    return pd.Series(reaction_rates)

In [ ]:
def get_reaction_branching_with_second_O2_addition(solution, initial_reactants):
    O2_concentration = solution.density_mole * .2
    branching = get_reaction_branching(solution, initial_reactants)
    reactants_with_O2 = initial_reactants.copy()
    reactants_with_O2.add('O2')
    with_O2_branching = get_reaction_branching(solution, reactants_with_O2) * O2_concentration
    return branching.append(with_O2_branching, verify_integrity=True)

In [ ]:
def get_peroxy_lifetime(solution, peroxy_name):
    """
    given the peroxy name, return the average lifetime of the peroxy
    which is given as the inverse of the sum of all unimolecular
    reactions in the network. 
    """
    return 

In [ ]:
image_dir = '../results'

In [ ]:
mechanism_file = '../data/mech_generation/pdep_merged/chem.cti'

In [ ]:
solution = ct.Solution(mechanism_file)
reactions = solution.reactions()

In [ ]:
n_xticks = 5
n_yticks = 5

In [ ]:
value_gradient = np.linspace(0,1,
                              256)

In [ ]:
color_list = np.zeros((256,4))
color_list[:,0] = value_gradient/2+.5
color_list[:,1] = value_gradient/2+.5
color_list[:,2] = value_gradient/2+.5
transparency_color_map = ListedColormap(color_list)

In [ ]:
black_color_map = ListedColormap(((0,0,0,1)))
white_color_map = ListedColormap(((1,1,1,1)))

In [ ]:
temperatures = np.linspace(180,1500,80)
pressures = np.logspace(3,7,80)

# fit multiple plots

In [ ]:
value_gradient = [0,.4,.75,.9,1]

In [ ]:
data_storage = {}
for s_index, surface in enumerate(['alpha', 'beta', 'gamma']):
    data_storage[surface] = {}
    for p_index, initial_pathway in enumerate(['alkyl','peroxy',]):
    #for p_index, initial_pathway in enumerate(['alkyl','peroxy','QOOH']):
        if surface == 'alpha':
            if initial_pathway == 'peroxy':
                initial_reactants = set(('aRO2',))
            elif initial_pathway == 'alkyl':
                initial_reactants = set(('O2','aR'))
            elif initial_pathway == 'QOOH':
                initial_reactants = set(('aQOOHg',))
        elif surface == 'beta':
            if initial_pathway == 'peroxy':
                initial_reactants = set(('bRO2',))
            elif initial_pathway == 'alkyl':
                initial_reactants = set(('O2','bR'))
            elif initial_pathway == 'QOOH':
                initial_reactants = set(('bQOOHa',))
        elif surface == 'gamma':
            if initial_pathway == 'peroxy':
                initial_reactants = set(('gRO2',))
            elif initial_pathway == 'alkyl':
                initial_reactants = set(('O2', 'gR'))
            elif initial_pathway == 'QOOH':
                initial_reactants = set(('gQOOHa',))
        main_path_matrix = pd.DataFrame(index=pressures,
                                 columns=temperatures,
                                 data=np.empty((len(temperatures),len(pressures))))
        fraction_main_path_matrix = pd.DataFrame(index=pressures,
                                 columns=temperatures,
                                 data=np.empty((len(temperatures),len(pressures))))

        for temperature in temperatures:
            for pressure in pressures:
                solution.TP = temperature, pressure
                if initial_pathway == 'QOOH':
                    paths = get_reaction_branching_with_second_O2_addition(solution,initial_reactants)
                else:
                    paths = get_reaction_branching(solution,initial_reactants)
                main_path = paths[paths == paths.max()].index[0]
                main_path_matrix.loc[pressure,temperature] = main_path
                fraction_main_path_matrix.loc[pressure,temperature] = paths[main_path] / paths.sum()

        data_one_plot = {}
        data_one_plot['main_path_matrix'] = main_path_matrix
        data_one_plot['fraction_main_path_matrix'] = fraction_main_path_matrix
        data_storage[surface][initial_pathway] = data_one_plot

In [ ]:
f, axes = plt.subplots(2,3,sharex=True,sharey=True,figsize = [12/.95,8],gridspec_kw={'wspace':.12,'hspace':0.2})

In [ ]:
print('Output below are the major pathways which contribute to the produced plots.\nThis allows you to add descriptive details in an SVG editor.')
for s_index, surface in enumerate(['alpha', 'beta', 'gamma']):
    for p_index, initial_pathway in enumerate(['alkyl','peroxy']):
        data_one_plot = data_storage[surface][initial_pathway]
        main_path_matrix = data_one_plot['main_path_matrix'] 
        fraction_main_path_matrix = data_one_plot['fraction_main_path_matrix']

        ax = axes[p_index][s_index]
        ax.set_xticks(np.linspace(0,len(temperatures)-1,n_xticks))
        ax.set_yticks(np.linspace(0,len(pressures)-1,n_yticks))
        ax.set_xticklabels([int(label) for label in np.linspace(temperatures.min(),temperatures.max(),n_xticks)])
        ax.set_yticklabels(['10$^{{{0}}}$'.format(int(np.log10(label))) for label in np.logspace(np.log10(pressures.min()),np.log10(pressures.max()),n_yticks)])
        if p_index == 1:
        #if p_index == 2:
            ax.set_xlabel('temperature (K)')
        if s_index == 0:
            ax.set_ylabel('pressure (Pa)')
        contour_output = ax.contourf(fraction_main_path_matrix,cmap=transparency_color_map,antialiased=True,levels=value_gradient, alpha=1)
        for rxn_index in np.unique(main_path_matrix):
            zero_one_matrix = np.zeros(main_path_matrix.shape)
            zero_one_matrix[main_path_matrix == rxn_index] = 1
            smooth_matrix = gaussian_filter(zero_one_matrix, 2)
            ax.contour(smooth_matrix,cmap=black_color_map,antialiased=True,levels=[.5])
            
        #make title
        if surface == 'alpha':
            title = u'α'
            if initial_pathway == 'QOOH':
                title += u'QOOHγ (+ O$_2$)'
        elif surface == 'beta':
            title = u'β'
            if initial_pathway == 'QOOH':
                title += u'QOOHα (+ O$_2$)'
        elif surface == 'gamma':
            title = u'γ'
            if initial_pathway == 'QOOH':
                title += u'QOOHα (+ O$_2$)'
        if initial_pathway == 'alkyl':
            title += 'R + O$_2$'
        elif initial_pathway == 'peroxy':
            title += 'RO$_2$'
        ax.set_title(title)
        print('########################    '+surface+'    '+initial_pathway)
        for rxn_index in np.unique(main_path_matrix):
            print('##########    '+reactions[int(rxn_index)].equation)
            main_path_df = pd.DataFrame(columns = temperatures, index=pressures, data=main_path_matrix)
            main_path_df_bool = main_path_df[main_path_df == rxn_index]
            pressures_of_path = main_path_df_bool.dropna(0,'all').index
            temps_of_path = main_path_df_bool.dropna(1,'all').columns
            print('temp range: ({0}, {1})'.format(min(temps_of_path), max(temps_of_path)))
            print('pres range: ({0}, {1})'.format(min(pressures_of_path), max(pressures_of_path)))

f.subplots_adjust(right=0.9)
cbar_ax = f.add_axes([0.91, .25, 0.06, 0.5])
cbar = f.colorbar(contour_output, cax=cbar_ax)
cbar_ax.set_axis_off()

In [ ]:
cbar_ax.annotate('product\nmixture',(.5,.125),xycoords='axes fraction',va='center', ha='center')
cbar_ax.annotate('major\nsecondary\nproducts',(.5,.375),xycoords='axes fraction',va='center', ha='center')
cbar_ax.annotate('minor\nsecondary\nproducts',(.5,.625),xycoords='axes fraction',va='center', ha='center')
cbar_ax.annotate('nearly\npure',(.5,.875),xycoords='axes fraction',va='center', ha='center')

In [ ]:
f.savefig(os.path.join(image_dir,'branching_6_plots.svg'),bbox_inches='tight')

# determine amount reversing

For the pdep reactions, a certain amount goes back to R + O2 which is not included in the Chemkin rates, since it is not a reaction, however it would be useful to know when there is another reaction competing for R radical.

To do this, we take the sum of all rates consuming R + O2 and divide it by the high p rate from high-p kinetics

In [ ]:
mechanism_file_high_p = '../data/mech_generation/high_p_merged/chem.cti'
solution_high_p = ct.Solution(mechanism_file_high_p)

In [ ]:
mechanism_file = '../data/mech_generation/pdep_merged/chem.cti'
solution = ct.Solution(mechanism_file)

### all three - linear

In [ ]:
value_gradient = np.linspace(0,1,21)

In [ ]:
f, axes = plt.subplots(1,3,sharex=True,sharey=True,figsize = [12,4],gridspec_kw={'wspace':.12,'hspace':0.2})

In [ ]:
for s_index, surface in enumerate(['alpha', 'beta', 'gamma']):
    if surface == 'alpha':
        initial_reactants = set(('O2','aR'))
    elif surface == 'beta':
        initial_reactants = set(('O2','bR'))
    elif surface == 'gamma':
        initial_reactants = set(('O2', 'gR'))
    
    fraction_reacts = pd.DataFrame(index=pressures,
                             columns=temperatures,
                             data=np.empty((len(temperatures),len(pressures))))

    for temperature in temperatures:
        for pressure in pressures:
            solution.TP = temperature, pressure
            solution_high_p.TP = temperature, pressure
            output = get_reaction_branching(solution, initial_reactants)
            output_high_p= get_reaction_branching(solution_high_p, initial_reactants)
            fraction_reacts.loc[pressure,temperature] = (output.sum() / output_high_p).values[0]
    
    ax = axes[s_index]
    ax.set_xticks(np.linspace(0,len(temperatures)-1,n_xticks))
    ax.set_yticks(np.linspace(0,len(pressures)-1,n_yticks))
    ax.set_xticklabels([int(label) for label in np.linspace(temperatures.min(),temperatures.max(),n_xticks)])
    ax.set_yticklabels(['10$^{{{0}}}$'.format(int(np.log10(label))) for label in np.logspace(np.log10(pressures.min()),np.log10(pressures.max()),n_yticks)])
    ax.set_xlabel('temperature (K)')
    if s_index == 0:
        ax.set_ylabel('pressure (Pa)')
    contour_output = ax.contourf(fraction_reacts,cmap=transparency_color_map,antialiased=True,alpha=1, levels=value_gradient)
    add_lines_to_contourf(contour_obj=contour_output)
    #make title
    if surface == 'alpha':
        title = u'α'
    elif surface == 'beta':
        title = u'β'
    elif surface == 'gamma':
        title = u'γ'
    title += 'R + O$_2$'
    ax.set_title(title)

cbar_ax = f.add_axes([0.925, 0.15, 0.02, 0.7])
cbar = f.colorbar(contour_output, cax=cbar_ax)
cbar_ax.set_ylabel('fraction of excited RO$_2$ not reforming R + O$_2$')
#cbar.set_ticks([0,-.5,-1,-1.5,-2,])


In [ ]:
f.savefig(os.path.join(image_dir,'non_reaction_fraction.svg'),bbox_inches='tight')

In [ ]:
f.savefig(os.path.join(image_dir,'non_reaction_fraction.pdf'),bbox_inches='tight')